## 02 Data Processing

#### Import relevant libraries

In [68]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

#### Load raw data

In [69]:
input_dir = os.path.join('..', 'data', 'original')
exp_df = pd.read_csv(os.path.join(input_dir, 'expenditure_bills_burden.csv'))
hui_df = pd.read_csv(os.path.join(input_dir, 'housing_units_income.csv'))
aei_df = pd.read_csv(os.path.join(input_dir, 'assets_earnings_investments.csv'))

In [70]:
exp_df.head()

,parent_name,utility_name,respondent_id,year,percent_AMI,ownership,electricity_gas_other,technology,expenditure,bill,burden
0,Southern Co.,Alabama Power Co.,2.0,2020,0-30%,owner,Electricity,adjustment,9.276270e+06,8.394506,0.009773
1,Southern Co.,Alabama Power Co.,2.0,2020,0-30%,owner,Electricity,distribution,3.017570e+07,27.307328,0.031792
2,Southern Co.,Alabama Power Co.,2.0,2020,0-30%,owner,Electricity,hydro,4.312818e+06,3.902860,0.004544
3,Southern Co.,Alabama Power Co.,2.0,2020,0-30%,owner,Electricity,nuclear,1.912960e+07,17.311221,0.020154
4,Southern Co.,Alabama Power Co.,2.0,2020,0-30%,owner,Electricity,other,1.578084e+07,14.280782,0.016626


In [71]:
hui_df.head()

,parent_name,utility_name,respondent_id,year,percent_AMI,ownership,housing_units,income
0,Southern Co.,Alabama Power Co.,2.0,2020,0-30%,owner,92086.710664,9.491514e+08
1,Southern Co.,Alabama Power Co.,2.0,2020,0-30%,renter,126159.321739,1.238083e+09
2,Southern Co.,Alabama Power Co.,2.0,2020,100%+,owner,521425.299562,6.049073e+10
3,Southern Co.,Alabama Power Co.,2.0,2020,100%+,renter,111421.757248,9.494001e+09
4,Southern Co.,Alabama Power Co.,2.0,2020,30-60%,owner,108469.889403,2.541532e+09


In [72]:
aei_df.head()

,parent_name,utility_name,respondent_id,year,asset,sub_asset,asset_value,earnings_value,investment_value
0,"American Electric Power Co., Inc.",AEP Generating Co.,1.0,2020,other,AROs,-15405378.0,-0.0,NaN
1,"American Electric Power Co., Inc.",AEP Generating Co.,1.0,2020,other,electric_plant_held_for_future_use,1034099.0,0.0,NaN
2,"American Electric Power Co., Inc.",AEP Generating Co.,1.0,2020,other,electric_plant_leased_to_others,155616036.0,0.0,NaN
3,"American Electric Power Co., Inc.",AEP Generating Co.,1.0,2020,other,general_plant,427318.0,0.0,1846.0
4,"American Electric Power Co., Inc.",AEP Generating Co.,1.0,2020,other,intangible_plant,724784.0,0.0,NaN


#### Formatting data

(a) Rename columns

In [73]:
exp_df = exp_df.rename({'percent_AMI': 'percent_ami'}, axis='columns')
hui_df = hui_df.rename({'percent_AMI': 'percent_ami'}, axis='columns')

(b) Data type formatting

In [74]:
exp_df.dtypes

parent_name               object
utility_name              object
respondent_id            float64
year                       int64
percent_ami               object
ownership                 object
electricity_gas_other     object
technology                object
expenditure              float64
bill                     float64
burden                   float64
dtype: object

In [75]:
hui_df.dtypes

parent_name       object
utility_name      object
respondent_id    float64
year               int64
percent_ami       object
ownership         object
housing_units    float64
income           float64
dtype: object

In [76]:
aei_df.dtypes

parent_name          object
utility_name         object
respondent_id       float64
year                  int64
asset                object
sub_asset            object
asset_value         float64
earnings_value      float64
investment_value    float64
dtype: object

#### Join datasets

Prepare expenditure_bills_burden.csv data frame

In [77]:
# make a copy
exp_prepared_df = exp_df.copy()

In [78]:
# groupby utility x year
exp_prepared_df = exp_prepared_df.groupby(['respondent_id', 'year', 'technology', 'electricity_gas_other'])\
    .agg({'bill': ['sum'], 'expenditure': ['sum']})\
    .reset_index()

Prepare housing_units_income.csv data frame

In [79]:
# make a copy
hui_prepared_df = hui_df.copy()

In [80]:
# drop repeated columns
hui_prepared_df = hui_prepared_df.drop(columns=[
    'parent_name', 
    'utility_name', 
    'ownership',
])

In [81]:
# groupby utility x year
hui_prepared_df = hui_prepared_df.groupby(['respondent_id', 'year'])\
    .agg({'housing_units': ['sum'], 'income': ['sum']})\
    .reset_index()

Prepare assets_earnings_income.csv data frame

In [82]:
# make a copy
aei_prepared_df = aei_df.copy()

In [83]:
# drop repeated columns
aei_prepared_df = aei_prepared_df.drop(columns=[
    'parent_name', 
    'utility_name', 
])

In [84]:
# groupby utility x year
aei_prepared_df = aei_prepared_df.groupby(['respondent_id', 'year', 'asset', 'sub_asset'])\
    .agg({'asset_value': ['sum'], 'earnings_value': ['sum'], 'investment_value': ['sum']})\
    .reset_index()

In [85]:
hui_prepared_df.head()

,respondent_id,year,housing_units,income
,,,sum,sum
0,2.0,2005,1177707.0,6.002705e+10
1,2.0,2006,1189597.0,6.287944e+10
2,2.0,2007,1202491.0,6.652127e+10
3,2.0,2008,1212244.0,6.973237e+10
4,2.0,2009,1228000.0,6.732521e+10


Join datasets into one dataframe

In [86]:
raw_df = pd.merge(exp_prepared_df, hui_prepared_df, how='inner', on=['respondent_id', 'year'])
raw_df = pd.merge(raw_df, aei_prepared_df, how='inner', on=['respondent_id', 'year'])
raw_df.head()

/home/julian/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,respondent_id,year,technology,electricity_gas_other,bill,expenditure,housing_units,income,asset,sub_asset,asset_value,earnings_value,investment_value
,,,,,sum,sum,sum,sum,,,sum,sum,sum
0,2.0,2005,Gas,Gas,375.031464,5.801132e+08,1177707.0,6.002705e+10,distribution,distribution,2.662308e+09,1.353981e+08,256240572.0
1,2.0,2005,Gas,Gas,375.031464,5.801132e+08,1177707.0,6.002705e+10,hydro,hydro,6.306366e+08,3.207254e+07,6223361.0
2,2.0,2005,Gas,Gas,375.031464,5.801132e+08,1177707.0,6.002705e+10,nuclear,nuclear,8.835976e+08,4.493748e+07,46613178.0
3,2.0,2005,Gas,Gas,375.031464,5.801132e+08,1177707.0,6.002705e+10,other,AROs,-4.462678e+08,-2.269602e+07,0.0
4,2.0,2005,Gas,Gas,375.031464,5.801132e+08,1177707.0,6.002705e+10,other,construction_work_in_progress,-4.035594e+05,-2.052398e+04,0.0


#### Data cleaning

(a) Impute values for missing data

(b) Remove probable outliers

(c) Compute consumer energy burden

#### Save processed_data file

In [87]:
output_dir = os.path.join('..', 'data', 'processed')

In [89]:
raw_df.to_csv(os.path.join(output_dir, 'processed_data.csv'))